In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge
from scipy import stats
import pickle

# Load necessary tables

In [ ]:
%%bigquery product
SELECT * FROM full_insurance_data.ProductType

In [ ]:
%%bigquery Disease
SELECT
    HP.ID,
    HP.Health_Condition_ID,
    HP.Hypertension,
    HP.High_Cholesterol,
    HP.CoronaryHeartDisease,
    HP.Angina,
    HP.HeartAttack,
    HP.Stroke,
    HP.Asthma,
    HP.Cancer,
    HP.Prediabetes,
    HP.GestationalDiabetes,
    HP.COPD,
    HP.Arthritis,
    HP.Dementia,
    HP.Anxiety_Disorder,
    HP.Depression,
    HP.Epilepsy,
    CC.Chronic_Fatigue_Syndrome
FROM
    full_insurance_data.Health_Problem AS HP
JOIN
    full_insurance_data.Current_Conditions AS CC
ON
    HP.ID = CC.ID

In [ ]:
# List of disease columns to check
disease_columns = [
    'Hypertension', 'High_Cholesterol', 'CoronaryHeartDisease', 'Angina',
    'HeartAttack', 'Stroke', 'Asthma', 'Cancer', 'Prediabetes',
    'GestationalDiabetes', 'COPD', 'Arthritis', 'Dementia',
    'Anxiety_Disorder', 'Depression', 'Epilepsy', 'Chronic_Fatigue_Syndrome'
]

# Function to check if any of the diseases is equal to 1 for a row
def has_disease(row):
    return any(pd.notna(row[col]) and row[col] == 1 for col in disease_columns)

# Create the 'disease' variable based on the custom function
Disease['disease'] = Disease.apply(has_disease, axis=1)

In [ ]:
%%bigquery Current_Condition
SELECT
    ID, 
    Health_Condition_ID,
    Weight,
    Height,
    Pregnant,
    Health_WeakImmune,
    (Weight * 0.45359237) / POWER((Height * 0.0254), 2) AS BMI
FROM
    full_insurance_data.Current_Conditions

In [ ]:
%%bigquery Lifestyle
SELECT *
FROM full_insurance_data.Alcohol AS A
JOIN full_insurance_data.smoking AS S
ON A.ID = S.ID
JOIN full_insurance_data.activity AS Act
ON A.ID = Act.ID

In [ ]:
%%bigquery demographics
SELECT ID, Urban_Rural,Region,Gender,Age,Race,Education,Num_Fam_Adult,Num_Fam_Kid, Current_MaritalStatus,Citizenship,JobYN,Housing
FROM full_insurance_data.Demographic

# Preprocessing

merged_df = demographics.merge(Current_Condition, on='ID').merge(Disease, on='ID').merge(Lifestyle,on='ID').merge(product, on='ID')
merged_df

In [ ]:
merged_df.shape

In [ ]:
# Drop people without insurance 
merged_df = merged_df[merged_df['ProductType'] == 1]

# Replace NaN values in 'Premium' with the median value
median_value = merged_df['Premium'].median()
merged_df['Premium'] = merged_df['Premium'].fillna(median_value)
#add log premium
merged_df['LogPremium']=np.log(merged_df['Premium'])
#Add smoke yes/no variable - 1=Yes, 0=No, 2=Unknown
smoke_mapping={1:1,2:1, 3:0,4:0,5:2,9:2}
merged_df['smoke'] = merged_df['SMKCIGST_A'].map(smoke_mapping)

#merged_df

#check missing data
#remove those with NA premiums
merged_df['Chronic_Fatigue_Syndrome']=merged_df['Chronic_Fatigue_Syndrome'].fillna(8)
merged_df.loc[merged_df.Gender==1&merged_df.Pregnant,'Pregnant']=2
merged_df['Pregnant']=merged_df['Pregnant'].fillna(8)

merged_df.isnull().sum()

# Linear Regression - Prediction of Insurance Premium Depending of different variables

# dividing dataset into train and test
x = merged_df[['Gender', 'Age', 'BMI','Num_Fam_Adult','Health_WeakImmune','Num_Fam_Kid','SMKCIGST_A','DRKSTAT_A',
              'Hypertension', 'High_Cholesterol', 'CoronaryHeartDisease', 'Angina',
    'HeartAttack', 'Stroke', 'Asthma', 'Cancer', 
    'GestationalDiabetes', 'COPD', 'Arthritis', 'Dementia',
    'Anxiety_Disorder', 'Depression', 'Epilepsy', 'Chronic_Fatigue_Syndrome','Citizenship','Urban_Rural','Education']]
y = merged_df[['LogPremium']]

# Split 20% with test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
cat_cols = [
    'Hypertension', 'High_Cholesterol', 'CoronaryHeartDisease', 'Angina',
    'HeartAttack', 'Stroke', 'Asthma', 'Cancer', 
    'GestationalDiabetes', 'COPD', 'Arthritis', 'Dementia',
    'Anxiety_Disorder', 'Depression', 'Epilepsy', 'Chronic_Fatigue_Syndrome','Citizenship','Urban_Rural',
'Education']
for c in cat_cols:
    X_train[c]=X_train[c].astype("category")
    X_test[c]=X_test[c].astype("category")

In [ ]:
set(X_train['Gender'])

In [ ]:
#Linear regression
model = LinearRegression()
model.fit(X_train, y_train)
train_pred = model.predict(X_train)

# calculate the accuracy of the model by computing the R2 score between predicted and real values
r2_train = metrics.r2_score(y_train, train_pred)
spearman=print('R squared value : ', r2_train)

In [ ]:
# prediction on test data
test_pred =model.predict(X_test)
res = stats.spearmanr(y_test, test_pred)
# R squared value
r2_test = metrics.r2_score(y_test, test_pred)
print('R squared value : ', r2_test)
print('Spearman Rank : ', res.statistic)

In [ ]:
import statsmodels.api as sm

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#hyperperameter tuning
grid_vals = {'penalty': ['l1','l2'], 'C': [0.001,0.005,0.01,0.05,0.1,0.5,1,5]}
#lasso
lasso_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}
lasso_model = Lasso()
lasso_grid_search = GridSearchCV(lasso_model, lasso_param_grid, cv=5, scoring='neg_mean_squared_error')
lasso_grid_search.fit(X_train, y_train)
best_lasso_alpha = lasso_grid_search.best_params_['alpha']
best_lasso_model = lasso_grid_search.best_estimator_

# Evaluate Lasso model
lasso_predictions = best_lasso_model.predict(X_test)
r2_test = r2_score(y_test, lasso_predictions)
res=stats.spearmanr(y_test, lasso_predictions)
print('R squared value : ', r2_test)
print(f"Lasso Best Alpha: {best_lasso_alpha}")
print(f"Lasso Spearman Rank: {res.statistic}")
#ridge

ridge_param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

# Create Ridge regression model
ridge_model = Ridge()

# Perform grid search for Ridge regression
ridge_grid_search = GridSearchCV(ridge_model, ridge_param_grid, cv=5, scoring='neg_mean_squared_error')
ridge_grid_search.fit(X_train, y_train)

# Get best hyperparameters and corresponding model
best_ridge_alpha = ridge_grid_search.best_params_['alpha']
best_ridge_model = ridge_grid_search.best_estimator_

# Evaluate Ridge model
ridge_predictions = best_ridge_model.predict(X_test)
ridge_r2 = r2_score(y_test, ridge_predictions)
res=stats.spearmanr(y_test, ridge_predictions)
print(f"Ridge Best Alpha: {best_ridge_alpha}")
print(f"Ridge R2: {ridge_r2}")
print(f"Ridge Spearman Rank: {res.statistic}")

# Decision Tree - Regression: whether data collected about disease can increase the cost of insurance products

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
#drop id from merged_df for correaltion analysis - string
#merged_df.drop('Premium')
corr_analysis = merged_df.drop(['ID', 'Age','Health_Condition_ID_x', 'Weight',
                                'Height', 'Pregnant', 'Health_Condition_ID_y',
                                'disease', 'Life_Style_ID', 'DRKSTAT_A', 'ID_1', 'Life_Style_ID_1',
                                'SMKCIGST_A', 'ID_2', 'Life_Style_ID_2','Premium','LogPremium', 'BMI'], axis=1)

#categorical variables in list
cat_vars=['Urban_Rural','Region','Gender','Race','Education','Num_Fam_Adult','Num_Fam_Kid',
          'Current_MaritalStatus','Citizenship','JobYN','Housing','Health_WeakImmune',
          'Hypertension','High_Cholesterol','CoronaryHeartDisease','Angina','HeartAttack',
          'Stroke','Asthma','Cancer','Prediabetes','GestationalDiabetes','COPD',
          'Arthritis','Dementia','Anxiety_Disorder','Depression','Epilepsy',
          'Chronic_Fatigue_Syndrome','Walking','Sleeping','Eating','Meditation',
          'Yoga','Therapy','Dr_Visit','Coverage','ProductType','smoke']

# Perform one-hot encoding on the categorical variables
data_encoded = pd.get_dummies(corr_analysis, columns=cat_vars)
selected_columns = ['Age', 'BMI', 'LogPremium']
combined_corr_analysis = pd.concat([data_encoded, merged_df[selected_columns]], axis=1)
combined_corr_analysis

In [ ]:
# Calculate correlation matrix for the combined DataFrame
correlation_matrix = combined_corr_analysis.corr() 
correlation_matrix

In [ ]:
correlation_matrix['LogPremium'].abs()

In [ ]:
# Assuming 'data' is your DataFrame with variables and target variable
correlation_matrix = combined_corr_analysis.corr()
x=correlation_matrix['LogPremium'].abs()
correlation_with_target = correlation_matrix['LogPremium'].abs().sort_values(ascending=False)

# Select the top N features with the highest correlation coefficients (e.g., top 10)
top_n_features = correlation_with_target.head(11).index.tolist()

print("Top N features with the highest correlation to 'LogPremium':")
print(top_n_features)

In [ ]:
# Split the data into features (X) and target variable (y)
x = combined_corr_analysis[['Num_Fam_Adult_1', 'JobYN_1', 'Current_MaritalStatus_1', 'Current_MaritalStatus_5', 'Housing_2', 'Num_Fam_Kid_0', 'Num_Fam_Adult_3', 'Current_MaritalStatus_9', 'Housing_1', 'Citizenship_2']]
y = combined_corr_analysis[['LogPremium']]

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Create a Decision Tree Regressor model
dt_model = DecisionTreeRegressor()

# Train the model on the training data
dt_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = dt_model.predict(X_test)

In [ ]:
# Calculate mean squared error (MSE) and R-squared (R2) for evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
res=  stats.spearmanr(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print('Spearman Rank:', res.statistic)

In [ ]:
#serialize model:
trained_model = pickle.dumps(dt_model,'test.pickle')

In [ ]:
pickle.dump(dt_model, open(filename, 'wb'))